# Solution: Can your favourite handball team still win?

## A real world-example: HBL season 2017/18

The table after matchday 24 is the following.

| Rank | Team                   | Points |
|------|------------------------|--------|
|  1   | Rhein-Neckar Löwen     | 40     |
|  2   | Füchse Berlin          | 40     |
|  3   | SG Flensburg-Handewitt | 38     |
|  4   | TSV Hannover-Burgdorf  | 37     |
|  5   | SC Magdeburg           | 35     |
|  7   | THW Kiel               | 31     |
|  6   | MT Melsungen           | 31     |
|  8   | SC DHfK Leipzig        | 29     |
|  9   | HSG Wetzlar            | 23     |
| 10   | TBV Lemgo              | 21     |
| 11   | FRISCH AUF! Göppingen  | 20     |
| 12   | TSV GWD Minden         | 19     |
| 13   | HC Erlangen            | 16     |
| 14   | TVB 1898 Stuttgart     | 13     |
| 15   | VfL Gummersbach        | 12     |
| 16   | TuS N-Lübbecke         | 11     |
| 17   | TV 05/07 Hüttenberg    |  9     |
| 18   | Die Eulen Ludwigshafen |  7     |

<font color="blue"><b>First task:</b></font> Max's argument is not complete - just arguing that a team has enough remaining games to catch up with the *current* score of the leading team is not enough. Max checked that the two leading teams do not play each other, but this is not enough: Imagine that in the remaining season, the first three teams will all play each other once, then at least one of them will have 42 points! Thus, we conclude that a final decision cannot be made from seeing the current table only, there is a dependence on the remaining games that are to be played.

## The general setting

<font color="blue"><b>Second task:</b></font> An implementation of the function `can_they_win()` is given below.

In [ ]:
import networkx as nx

def can_they_win(current_points, remaining_games_graph, team):

    # calculate final points of the given team
    #   (assuming that they win all their games)
    team_matches = [e for e in remaining_games_graph.edges(keys = True) if team in e]
    final_points_team = current_points[team] + 2 * len(team_matches)
    
    # check if there is a team with strictly more points already
    for pts in current_points.values():
        if pts > final_points_team:
            return False

    # get matches not involving given team and remaining points
    other_matches = [e for e in remaining_games_graph.edges() if team not in e]
    num_other_matches = len(other_matches)
    remaining_other_points = 2 * num_other_matches
    
    # get list of all teams involved
    teams = current_points.keys()

    # generate flow digraph, add vertices and edges
    g = nx.DiGraph()
    g.add_nodes_from(["s", "t"] + other_matches + [t for t in teams if t != team])
    for m in other_matches:
        g.add_edge("s", m, capacity = 2)
        g.add_edge(m, m[0], capacity = 2)
        g.add_edge(m, m[1], capacity = 2)
    for t in current_points.keys():
        if t != team:
            g.add_edge(t, "t", capacity = final_points_team - current_points[t] - 1)

    # calculate max flow value
    val = nx.maximum_flow_value(g, "s", "t")

    # return whether max flow has desired value
    return val == remaining_other_points

## Testing the implementation

<font color="blue"><b>Third task:</b></font> Test if "TBV Lemgo" and "HSG Wetzlar" can become sole leaders at the end of the season, judging from the standings after matchday 24.

In [ ]:
import winningPossibilities_module as helpers

current_points = helpers.get_current_points(24)
remaining_games_graph = helpers.get_remaining_games_graph(24)

# Can "TBV Lemgo" be the sole leader at the end of the season?
team = "TBV Lemgo"
res = can_they_win(current_points, remaining_games_graph, team)
print(f"{team} can {'' if res else 'not'} be the sole leader after matchday 34.")

# Can "HSG Wetzlar" be the sole leader at the end of the season?
team = "HSG Wetzlar"
res = can_they_win(current_points, remaining_games_graph, team)
print(f"{team} can {'' if res else 'not '}be the sole leader after matchday 34.")

## Getting certificates

<font color="blue"><b>Fourth task:</b></font> Providing certificates if a team cannot win. It was shown in the lecture (see lecture notes) how such certificates can be obtained from minimum cuts in a graph that is essentially the auxiliary digraph used above (except for the fact that edge capacities of edges connecting a game to a team are set to values larger than 2): It is precisely the set of all teams in a minimum cut.

The returned certificates are (in case the answer is `False`) sets of teams such that at the end of the season, at least one of the teams in the set has the same number of points as the given team will, based only on the current points and games that are played among the teams in the returned set. From the discussion in the lecture, we know that if the answer is `False`, then such a set always exists, and we will find it.

In [ ]:
def can_they_win_certificate(current_points, remaining_games_graph, team):
    # Note: Comments only on what changed compared to previous implementation!
    
    team_matches = [e for e in remaining_games_graph.edges(keys = True) if team in e]
    final_points_team = current_points[team] + 2 * len(team_matches)
    
    # if there is a team with strictly more points already, return false and the team
    for tm, pts in current_points.items():
        if pts > final_points_team:
            return (False, set(tm))

    other_matches = [e for e in remaining_games_graph.edges() if team not in e]
    num_other_matches = len(other_matches)
    remaining_other_points = 2 * num_other_matches
    
    teams = set(current_points.keys())

    # the flow digraph now has capacity 3 on edges connecting matches to teams
    g = nx.DiGraph()
    g.add_nodes_from(["s", "t"] + other_matches + [t for t in teams if t != team])
    for m in other_matches:
        g.add_edge("s", m, capacity = 2)
        g.add_edge(m, m[0], capacity = 3)
        g.add_edge(m, m[1], capacity = 3)
    for t in current_points.keys():
        if t != team:
            g.add_edge(t, "t", capacity = final_points_team - current_points[t] - 1)

    # calculate min cut and its value
    val, partition = nx.minimum_cut(g, "s", "t")
    
    # Can the given team finish as sole leader?
    answer = val == remaining_other_points
    
    # If no, get certificate (teams in min s-t cut)
    certificate = None if answer else teams.intersection(partition[0] if "s" in partition[0] else partition[1])

    # return answer and certificate
    return (answer, certificate)

To get a certificate for the case of "TBV Lemgo", we run the following code.

In [ ]:
import winningPossibilities_module as helpers

current_points = helpers.get_current_points(24)
remaining_games_graph = helpers.get_remaining_games_graph(24)

# Can "TBV Lemgo" be the sole leader at the end of the season?
team = "TBV Lemgo"
(res, cert) = can_they_win_certificate(current_points, remaining_games_graph, team)
print(f"{team} can {'' if res else 'not'} be the sole leader after matchday 34.")
if not res:
    print(f"Certificate: {cert}")

We know that the certificate set is a set of teams among which at least one will have more points than "TBV Lemgo" at the end of the season. To explain this to your friend Max, you could show him the following.

Consider the following teams with their current points:

In [ ]:
sum = 0
nb_teams = 0
for t in cert:
    pts = current_points[t]
    print(f'"{t}": {pts} Points')
    sum += pts
    nb_teams += 1
print()
print(f"Total: {nb_teams} teams, {sum} points")

In the remaining season, there will be the following games among these teams:

In [ ]:
count = 0
for e in remaining_games_graph.edges(keys = True):
    if e[0] in cert and e[1] in cert:
        print(f"{e[0]} vs. {e[1]}")
        count += 1
print()
print(f"There are {count} games among the teams, with {2*count} points to be distributed.")

From this information, it is easy to see that at least one of these teams will have at least 41 points at the end of the season:

In [ ]:
print(f"The {nb_teams} teams listed above will on average have " +
      f"at least {(sum + 2*count)/nb_teams} points at the end of the season.")

Thus, "TBV Lemgo", who can reach at most 41 points, will definitely not be the sole leader at the end of the season.